In [ ]:
from simple_cysh import * # includes pandas as pd

In [ ]:
os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
# Fill in details on each section, enure no duplicates (one staff per section)
section_df = get_cysh_df('Section__c', ['Id', 'Name', 'School__c', 'Program__c', 'In_After_School__c', 'Intervention_Primary_Staff__c'], rename_id=True)
section_df.rename(columns={'Intervention_Primary_Staff__c':'Staff__c'}, inplace=True)
program_df = get_cysh_df('Program__c', ['Id', 'Name'], rename_id=True, rename_name=True)
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'Role__c'], rename_id=True, rename_name=True)

In [ ]:
# add program tpye to section__c
df = section_df.merge(program_df, how='left', on='Program__c')
# add staff 'Name' and 'Role__c' to section__c
df = df.merge(staff_df, how='left', on='Staff__c')
# filter
df = df.loc[~df['Program__c_Name'].isnull()]
df = df.loc[df['Program__c_Name'].str.contains('Tutoring')
            & (df['In_After_School__c']=='In School')]

In [ ]:
# sum student ToT by program, then merge back in to sections
student_section_df = get_cysh_df('Student_Section__c', ['Id', 'Section__c', 'Student__c'], rename_id=True)
ISR_df = get_cysh_df('Intervention_Session_Result__c', ['CreatedDate', 'Intervention_Session_Date__c',
                                                        'Amount_of_Time__c', 'Primary_Skill__c',
                                                        'Student_Section__c', 'Intervention_Session__c',
                                                        'Section_Name__c', 'Program__c', 'Program_Name__c'])

ISR_df = ISR_df.merge(student_section_df, how='left', on='Student_Section__c')
ISR_df['student_program'] = ISR_df['Student__c'] + '_' + ISR_df['Program__c']
ISR_df['Intervention_Session_Date__c'] = pd.to_datetime(ISR_df['Intervention_Session_Date__c'])
ISR_df = ISR_df.loc[ISR_df['Intervention_Session_Date__c'] < '2017-12-11']
# add ToT sum by student_program
ISR_df = ISR_df.join(ISR_df.groupby('student_program')['Amount_of_Time__c'].sum(), how='left', on='student_program', rsuffix='_YTD')

In [ ]:
# drop duplicates on student_section, reduce columns
ISR_df = ISR_df[['Student_Section__c', 'Section__c', 'Student__c', 'student_program', 'Amount_of_Time__c_YTD']].drop_duplicates('Student_Section__c')

In [ ]:
df = ISR_df.merge(df, on='Section__c')

In [ ]:
df.to_csv('time_on_task_2017-12-11.csv', index=False)

In [ ]:
# define active as students who received more than 200 mins tutoring?
# other factors to add at some point: Number of ACMs who contributed a meaningful amount of time